# 1. Nested cross-validation exercise
## Nested cross-validation for feature selection with nearest neighbors <br>
- Use Python 3 to program both a hyper-parameter selection method based on leave-one-out cross-validation and a nested leave-one-out cross-validation for estimating the prediction performance of models inferred with this automatic selection approach. Use base learning algorithm provided in the exercise, namely the "use_ith_feature" method, so that the value of the hyper-parameter i is automatically selected from the range from 1 to 100 of alternative values. The provided base learning algorithm "use_ith_feature" is 1-nearest neighbor that uses only the ith feature of the data given to it. The LOOCV based hyper-parameter selection procedure is supposed to select the best feature, e.g. the value of i, based on C-index evaluated with predictions obtained with leave-one-out cross-validation. A ready-made implementation of C-index is also provided in the exercise. In nested leave-one-out cross-validation, a C_index value is further evaluated on the predictions obtained from an outer leave-one-out cross-validation. During each round of this outer LOOCV, the whole feature selection process based on inner LOOCV is separately done and the selected feature is used for prediction for the test data point held out during that round of the outer LOOCV. Accordingly, the actual learning algorithm, whose prediction performance will be evaluated with nested CV, is the one that automatically selects the single best feature with leave-one-out cross-validation based model selection (see the lectures and the pseudo codes presented on them for more info on this interpretation).
- Note that since the hold-out set in LOOCV has only a single datum but C-index requires at least two data points. The solution in this exercise is to "pool" the predictions of all LOOCV rounds of a single LOOCV computation into an array of length of the data used in that LOOCV computation and then compute C-index on that array and the corresponding true outputs. This pooling approach, however, does have its weaknesses, since C-index computed from pooled LOOCV outputs may sometimes be a heavily biased estimator of the true C-index. This has been considered in detail in our previous research (and other group's too as seen in the references) that is available here:
http://dx.doi.org/10.1177/0962280218795190
where AUC, a special case of C-index, is considered. The study goes quite deep into the problem of AUC estimation with CV, and you can read it if you are interested about the research carried out in our laboratory, while EMLM course does not go that far and this year's exercise unfortunately still has this non-optimal pooling approach in use.
- Compare the C-index produced by nested leave-one-out CV with the result of ordinary leave-one-out CV with the best value of i e.g. the feature providing the highest LOOCV C-index, and show the C-index difference between the two methods.
- Use the provided implementation of the "use_ith_feature" learning algorithm and C-index functions in your exercise.

As a summary, for completing this exercise implement the following steps: 
_______________________________________________________________
#### 1. Use leave-one-out cross-validation for determining the optimal i-parameter for the data (X_alternative.csv, y_alternative.csv) from the set of possible values of i e.g. {1,...,100}. When you have found the optimal i, save the corresponding C-index (call it loo_c_index) for this parameter.
#### 2. Similarly, use nested leave-one-out cross-validation (leave-one-out both in outer and inner folds) for estimating the C-index (call it nloo_c_index) of the method that selects the best feature with leave-one-out approach. 
#### 3. Return both this notebook and as a PDF-file made from it in the exercise submit page. 
_______________________________________________________________

Remember to use the provided learning algorithm use_ith_feature and C-index functions in your implementation! 

## Import libraries

In [11]:
#In this cell import all libraries you need. For example: 
import numpy as np
import pandas as pd
from sklearn.model_selection import LeaveOneOut

## Provided functions 

In [2]:
"""
C-index function: 
- INPUTS: 
'y' an array of the true output values
'yp' an array of predicted output values
- OUTPUT: 
The c-index value
"""
def cindex(y, yp):
    n = 0
    h_num = 0 
    for i in range(0, len(y)):
        t = y[i]
        p = yp[i]
        for j in range(i+1, len(y)):
            nt = y[j]
            np = yp[j]
            if (t != nt): 
                n = n + 1
                if (p < np and t < nt) or (p > np and t > nt): 
                    h_num += 1
                elif (p == np):
                    h_num += 0.5
    return h_num/n

"""
Self-contained 1-nearest neighbor using only a single feature
- INPUTS: 
'X_train' a numpy matrix of the X-features of the train data points
'y_train' a numpy matrix of the output values of the train data points
'X_test' a numpy matrix of the X-features of the test data points
'i' the index of the feature to be used with 1-nearest neighbor
- OUTPUT: 
'y_predictions' a list of the output value predictions
"""
def use_ith_feature(X_train, y_train, X_test, i):
    y_predictions = []
    for test_ind in range(0, X_test.shape[0]):
        diff = X_test[test_ind, i] - X_train[:, i]
        distances = np.sqrt(diff * diff)
        sort_inds = np.array(np.argsort(distances), dtype=int)
        y_predictions.append(y_train[sort_inds[0]])
    return y_predictions


## Your implementation here

In [17]:
# In this cell implement the required tasks

X = np.genfromtxt('X_alternative.csv', delimiter = ',')
y = np.genfromtxt('y_alternative.csv')

In [12]:
loo = LeaveOneOut()


In [13]:
results = []

for i in range(100):
    yp = []
    
    for fold, (train_index, test_index) in enumerate(loo.split(X)):
        X_train, y_train, X_test = X[train_index], y[train_index], X[test_index]
        yp.append(use_ith_feature(X_train, y_train, X_test, i)[0])
        
    results.append(cindex(y,yp))
    i+=1

In [16]:
results

[0.3793103448275862,
 0.532183908045977,
 0.3563218390804598,
 0.5850574712643678,
 0.5597701149425287,
 0.5517241379310345,
 0.3850574712643678,
 0.5057471264367817,
 0.4471264367816092,
 0.5057471264367817,
 0.5183908045977011,
 0.5057471264367817,
 0.4471264367816092,
 0.4091954022988506,
 0.5149425287356322,
 0.32068965517241377,
 0.6011494252873564,
 0.5080459770114942,
 0.593103448275862,
 0.539080459770115,
 0.5298850574712644,
 0.49310344827586206,
 0.5551724137931034,
 0.4908045977011494,
 0.46551724137931033,
 0.5091954022988506,
 0.4540229885057471,
 0.535632183908046,
 0.5252873563218391,
 0.3448275862068966,
 0.4494252873563218,
 0.5172413793103449,
 0.4862068965517241,
 0.3620689655172414,
 0.5850574712643678,
 0.5505747126436782,
 0.3275862068965517,
 0.4011494252873563,
 0.5275862068965518,
 0.38045977011494253,
 0.40804597701149425,
 0.37471264367816093,
 0.32988505747126434,
 0.4850574712643678,
 0.542528735632184,
 0.5827586206896552,
 0.4114942528735632,
 0.55057471